# Parsimony calculations

In [1]:
import toytree
import toyplot
import numpy as np

### Notebook

In [660]:
import toytree

# an example five tip tree (with tip names alpha-epsilon)
tree = toytree.tree("(((alpha,delta),gamma),(beta,epsilon));")

# a single binary trait character state for each species
trait = {
    "alpha": 1,
    "beta": 0,
    "delta": 1,
    "epsilon": 0,
    "gamma": 1,
}

In [661]:
# sets a .trait attribute to each Node in trait dictionary
tree.set_node_data(
    feature="trait",
    mapping=trait,
    inplace=True,
)

# get and display the Node data as a dataframe
tree.get_node_data(['name', 'trait'])

,name,trait
0,alpha,1
1,delta,1
2,gamma,1
3,beta,0
4,epsilon,0
5,,
6,,
7,,
8,,


In [662]:
tree.draw(
    tree_style='s',
    use_edge_lengths=False, 
    tip_labels_style={"font-size": 18},
    node_labels="trait",
    node_colors=[
        toytree.color.COLORS1[i] if i !="" else "lightgrey" 
        for i in tree.get_node_data("trait")
    ],
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="285.0px" height="275.0px" viewBox="0 0 285.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="taad63d1b9b0842838d25b7ff05b22784"> 1 1 1 0 0 alpha delta gamma beta epsilon

In [663]:
from typing import Dict, Any

def fitch_parsimony_single(tree: toytree.ToyTree, trait: Dict[str,Any]) -> int:
    """Return Fitch parsimony score given a tree and single trait.

    For didactic purposes this function will also store a feature named 
    'fitch' to every Node which can be examined/visualized afterwards.
    
    Parameters
    ----------
    tree: ToyTree
        A tree with data 'state' on each tip Node rom which to
        calculate the Fitch parsimony score. 
    trait: Dict
        A dict mapping tip names to a discrete character state.
        
    Returns
    -------
    int
        The minimum changes required for trait data to evolve on this tree.
    """
    # counter to keep track of change events
    nchanges = 0

    # iterate over Nodes in idxorder (postorder sorted) traversal
    for node in tree:

        # leaves are visited first, and converted to a set type
        if node.is_leaf():
            node.fitch = set([trait[node.name]])

        # internal Nodes examine the sets of their children's states
        if not node.is_leaf():

            # check for shared (intersecting) states
            shared = set.intersection(*(i.fitch for i in node.children))

            # if any states are shared then ancestor inherits this state
            if shared:
                node.fitch = shared

            # if none shared, then store the union and increment counter
            else:
                node.fitch = set.union(*(i.fitch for i in node.children))
                nchanges += 1
    return nchanges

In [664]:
fitch_parsimony_single(tree, trait)

1

In [665]:
c, a, m = tree.draw(
    width=500, 
    use_edge_lengths=False,
    node_sizes=25, 
    node_markers='r2x1',
    node_mask=False,
    node_colors="lightgrey",
    node_labels="fitch", 
    node_labels_style={'font-size': 15},
    tip_labels_style={'font-size': 15, '-toyplot-anchor-shift': 30}
);
c.style['background-color'] = 'white'
import toyplot.svg
toyplot.svg.render(c, "./fig-6.3.svg")
c

### Rooting

In [666]:
# iterate over each edge that can become new root
for node in tree:
    if not node.is_root():
        # get a copy of tree rooted on new edge (selected by Node below)
        new_rooted_tree = tree.root(node)

        # print the parsimony score of this new tree
        score = fitch_parsimony_single(new_rooted_tree, trait)
        clade = node.get_leaf_names()
        print(f'score={score} when rooted on {clade}')

score=1 when rooted on ['alpha']
score=1 when rooted on ['delta']
score=1 when rooted on ['gamma']
score=1 when rooted on ['beta']
score=1 when rooted on ['epsilon']
score=1 when rooted on ['alpha', 'delta']
score=1 when rooted on ['alpha', 'delta', 'gamma']
score=1 when rooted on ['beta', 'epsilon']


In [667]:
new_rooted_tree = tree.root(5)
fitch_parsimony_single(new_rooted_tree, trait)

c, a, m = new_rooted_tree.draw(
    width=500, 
    use_edge_lengths=False,
    node_sizes=25, 
    node_markers='r2x1',
    node_mask=False,
    node_colors="lightgrey",
    node_labels="fitch", 
    node_labels_style={'font-size': 15},
    tip_labels_style={'font-size': 15, '-toyplot-anchor-shift': 30}
);
c.style['background-color'] = 'white'
import toyplot.svg
toyplot.svg.render(c, "./fig-6.4.svg")
c

### Alternate trees

In [668]:
# an example five tip tree (with tip names alpha-epsilon)
tree2 = toytree.tree("(((alpha,beta),gamma),(delta,epsilon));")

In [669]:
fitch_parsimony_single(tree2, trait)

2

In [670]:
c, a, m = tree2.draw(
    width=500, 
    use_edge_lengths=False,
    node_sizes=25, 
    node_markers='r2x1',
    node_mask=False,
    node_colors="lightgrey",
    node_labels="fitch", 
    node_labels_style={'font-size': 15},
    tip_labels_style={'font-size': 15, '-toyplot-anchor-shift': 30}
)
c.style['background-color'] = 'white'
import toyplot.svg
toyplot.svg.render(c, "./fig-6.5.svg")
c

### Multiple traits

In [671]:
data.iloc[:, 3]

alpha      1
beta       0
gamma      0
delta      1
epsilon    1
Name: t3, dtype: int64

In [708]:
a = np.zeros((100, 100))
b = pd.DataFrame(a)

In [709]:
%%timeit
a[50, 50]

86.9 ns ± 1.5 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [723]:
%%timeit
b.loc[50, 50]

5.3 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [728]:
%%timeit
names.index(10)

157 ns ± 2.18 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [739]:
data.loc[tree.get_tip_labels()]

,0,1,2
alpha,0,0,0
delta,1,0,0
gamma,1,0,1
beta,0,1,1
epsilon,0,1,0


In [748]:
from typing import Tuple

def preprocess_matrix(data: pd.DataFrame) -> Tuple[int, np.ndarray, pd.DataFrame]:
    """Return a score and dict of sites to be processed.

    This func will remove invariant sites and sites with only a single
    autapomorphy to get a reduced DataFrame with only unique patterns. 

    Parameters
    ----------
    data: pandas.DataFrame
        A pandas dataframe character matrix with taxon names as index.

    Returns
    -------
    Tuple 
        A 3-part tuple with (score, counts, data). The score is an int 
        for the number of singleton sites; the counts is an array of ints
        for the number of occurrence of each unique site pattern; the data
        is a DataFrame with the unique site patterns.
    """
    # get data as a numpy array
    arr = np.array(data)

    # get int array counting how many samples match the state of 
    # the first sample in each column. We will use this for filtering.
    counts = np.sum(arr == arr[0], axis=0)

    # get boolean array of invariant columns, i.e., all items match the first.
    invariant = counts == arr.shape[0]

    # get boolean array of singleton columns, i.e., nmaching is 1 or N-1
    singletons = (counts == 1) | (counts == arr.shape[0] - 1)

    # combine boolean masks and filter columns from data
    arr = arr[:, np.invert(invariant | singletons)]

    # count the number of singleton sites
    score = singletons.sum()

    # get unique site patterns and their counts
    arr, counts = np.unique(arr, axis=1, return_counts=True)

    # find columns that are the same PATTERN as others, i.e., recode patterns
    # such that [0, 1, 1, 2] == [1, 2, 2, 0] == [2, 0, 0, 1], etc.
    uniq_dict = {}
    for cidx in range(arr.shape[1]):

        # get a column of data. e.g., [1, 1, 0, 1, 2]
        col = arr[:, cidx]

        # get index of first occurrence of each unique base, e.g., [0, 2, 4],
        _, idxs = np.unique(col, return_index=True)

        # get order of indices, e.g., [0, 1, 2]
        order = np.argsort(idxs).tolist()

        # get site as ordered index of unique patterns, e.g., [0, 0, 1, 0, 2]
        site = tuple((order.index(i) for i in col))

        # record each uniq pattern, or add to it if already mirrored.
        if site not in uniq_dict:
            uniq_dict[site] = counts[cidx]
        else:
            uniq_dict[site] += counts[cidx]

    # convert unique patterns and counts back into a dataframe and array
    arr = np.column_stack(list(uniq_dict))
    data = pd.DataFrame(data=arr, index=data.index)
    counts = np.array(list(uniq_dict.values()))

    # return the score, counts, and data
    return score, counts, data

In [749]:
def fitch_parsimony(tree: toytree.ToyTree, data: pd.DataFrame) -> int:
    """Return Fitch parsimony score given a tree and single trait.

    For didactic purposes this function will also store a feature named 
    'fitch' to every Node which can be examined/visualized afterwards.
    
    Parameters
    ----------
    tree: ToyTree
        A tree on which to count state changes.
    data: DataFrame
        A dataframe with sample names from the tree as the index.
        
    Returns
    -------
    int
        The minimum changes required for trait data to evolve on this tree.
    """
    # reduce matrix to unique sites and counts, and get initial score
    score, counts, data = preprocess_matrix(data)

    # sort data to tip idxorder and convert to array (faster indexing)
    data = data.loc[tree.get_tip_labels()].values

    # iterate over each column index of data
    for cidx in range(data.shape[1]):

        # iterate over Nodes in idxorder (postorder sorted) traversal
        for node in tree:

            # leaves are visited first, and converted to a set type
            if node.is_leaf():
                char = data[node.idx, cidx]
                node._fitch = set([char])

            # internal Nodes examine the sets of their children's states
            if not node.is_leaf():

                # check for shared (intersecting) states
                shared = set.intersection(*(i._fitch for i in node.children))

                # if any states are shared then ancestor inherits this state
                if shared:
                    node._fitch = shared

                # if none shared, then store the union and increment counter
                else:
                    node._fitch = set.union(*(i._fitch for i in node.children))
                    score += counts[cidx]
    return score

In [752]:
fitch_parsimony(tree, data)

5

In [704]:
from typing import Tuple

def preprocess_matrix(data: pd.DataFrame) -> Tuple[int, np.ndarray, pd.DataFrame]:
    """Return a score and dict of sites to be processed.
    
    This func will remove invariant sites and sites with only a single
    autapomorphy, and will find and return only unique site patterns. 

    Parameters
    ----------
    data: pandas.DataFrame
        A pandas dataframe character matrix with taxon names as index.

    Returns
    -------
    Dict
        A dict with a key 'score' for the number of singleton sites, and a
        key 'data' with an array of unique site patterns, and a key 'counts'
        for the number of occurrences of each unique site pattern.
    """
    # get data as a numpy array
    arr = np.array(data)

    # get int array counting how many samples match the state of 
    # the first sample in each column. We will use this for filtering.
    counts = np.sum(arr == arr[0], axis=0)
    
    # get boolean array of invariant columns, i.e., all items match the first.
    invariant = counts == arr.shape[0]
    
    # get boolean array of singleton columns, i.e., nmaching is 1 or N-1
    singletons = (counts == 1) | (counts == arr.shape[0] - 1)
    
    # combine boolean masks and filter columns from data
    arr = arr[:, np.invert(invariant | singletons)]

    # count the number of singleton sites
    score = singletons.sum()

    # get unique site patterns and their counts
    arr, counts = np.unique(arr, axis=1, return_counts=True)

    # find columns that are the same PATTERN as others, i.e., recode patterns
    # such that [0, 1, 1, 2] == [1, 2, 2, 0] == [2, 0, 0, 1], etc.
    uniq_dict = {}
    for cidx in range(arr.shape[1]):
        
        # get a column of data. e.g., [1, 1, 0, 1, 2]
        col = arr[:, cidx]
        print(col)
        
        # get index of first occurrence of each unique base, e.g., [0, 2, 4],
        _, idxs = np.unique(col, return_index=True)
        
        # get order of indices, e.g., [0, 1, 2]
        order = np.argsort(idxs).tolist()
        
        # get site as ordered index of unique patterns, e.g., [0, 0, 1, 0, 2]
        site = tuple((order.index(i) for i in col))
        
        # record the first time pattern is seen
        if site not in uniq_dict:
            uniq_dict[site] = counts[cidx]

        # increment following times
        else:
            uniq_dict[site] += counts[cidx]
    
    # convert unique patterns and counts back into a dataframe and array
    data = pd.DataFrame(data=np.column_stack(list(uniq_dict)), index=data.index)
    counts = np.array(list(uniq_dict.values()))

    # return the score, counts, and data
    return score, counts, data

In [705]:
score, counts, data = preprocess_matrix(data)

print(f"score={score}")
print(counts)
print(data)

[0 0 1 1 0]
[0 1 0 0 1]
[0 1 1 0 0]
score=0
[1 1 1]
         0  1  2
alpha    0  0  0
beta     0  1  1
gamma    1  0  1
delta    1  0  0
epsilon  0  1  0


In [755]:
%%timeit
a[50, 50]

90.7 ns ± 2.88 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [756]:
%%timeit
b.iloc[50, 50]

20.1 µs ± 447 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


,0,1,2
alpha,0,0,0
beta,0,1,1
gamma,1,0,1
delta,1,0,0
epsilon,0,1,0


In [624]:
np.unique([2, 0, 1, 1, 2], return_index=True, return_inverse=True)

(array([0, 1, 2]), array([1, 2, 0]), array([2, 0, 1, 1, 2]))

In [625]:
np.unique([0, 0, 2, 1, 2], return_index=True, return_inverse=True)

(array([0, 1, 2]), array([0, 3, 2]), array([0, 0, 2, 1, 2]))

In [606]:
preprocess_matrix(data)

{'score': 1,
 'traits': {(0, 0, 1, 1, 0): 1, (0, 1, 0, 0, 1): 2, (0, 1, 1, 0, 0): 2}}

In [684]:
def fitch_parsimony(tree: toytree.ToyTree, data: pd.DataFrame) -> int:
    """Return Fitch parsimony score given a tree and single trait.

    For didactic purposes this function will also store a feature named 
    'fitch' to every Node which can be examined/visualized afterwards.
    
    Parameters
    ----------
    tree: ToyTree
        A tree on which to count state changes.
    data: DataFrame
        A dataframe with sample names from the tree as the index.
        
    Returns
    -------
    int
        The minimum changes required for trait data to evolve on this tree.
    """
    # reduce matrix to unique sites and get initial score
    pre = preprocess_matrix(data)
    score = pre["score"]
    traits = pre["traits"]

    # a dict to translate names into numeric order in traits tuples
    name_to_order = dict((j, i) for (i, j) in enumerate(data.index))    

    # iterate over each unique trait
    for trait, count in traits.items():

        # iterate over Nodes in idxorder (postorder sorted) traversal
        for node in tree:

            # leaves are visited first, and converted to a set type
            if node.is_leaf():
                idx = name_to_order[node.name]
                node._fitch = set([trait[idx]])

            # internal Nodes examine the sets of their children's states
            if not node.is_leaf():

                # check for shared (intersecting) states
                shared = set.intersection(*(i._fitch for i in node.children))

                # if any states are shared then ancestor inherits this state
                if shared:
                    node._fitch = shared

                # if none shared, then store the union and increment counter
                else:
                    node._fitch = set.union(*(i._fitch for i in node.children))
                    score += count
    return score

In [600]:
tree = toytree.tree("(((alpha,delta),gamma),(beta,epsilon));")

In [601]:
fitch_parsimony(tree, data)

9

In [570]:
trait

{'alpha': 1, 'beta': 0, 'delta': 1, 'epsilon': 0, 'gamma': 1}

In [558]:
dict(zip(data.index, (0, 0, 1, 1, 0)))

{'alpha': 0, 'beta': 0, 'gamma': 1, 'delta': 1, 'epsilon': 0}

In [512]:
a, b, c = np.unique([3, 3, 1, 1, 0], return_inverse=True, return_index=True)
a, b, c

(array([0, 1, 3]), array([4, 2, 0]), array([2, 2, 1, 1, 0]))

In [520]:
a, b, c = np.unique([1, 1, 0, 0, 3], return_inverse=True, return_index=True)
a, b, c

(array([0, 1, 3]), array([2, 0, 4]), array([1, 1, 0, 0, 2]))

In [525]:
np.argsort(b).tolist()

[1, 0, 2]

In [521]:
order = np.argsort(b)
for x, y in zip(a, order):
    c[c == y] = x   
c

array([1, 1, 1, 1, 3])

In [515]:
order = np.argsort(b)
for uidx, sidx in enumerate(order):
    bb = b[sidx]
    pos = c[bb]
    print(uidx, sidx, bb, pos)

0 1 0 1
1 0 2 0
2 2 4 2


In [458]:
{idx: [1, 1, 0, 0, 2][base] for idx, base in enumerate(sorted([2, 0, 4]))}

{0: 1, 1: 0, 2: 2}

In [476]:
_, idx, inv = np.unique([3, 3, 1, 1, 0], return_inverse=True, return_index=True)
dd = {i: inv[base] for i, base in enumerate(sorted(idx))}
#[dd[i] for i in inv]
_, idx, inv

(array([0, 1, 3]), array([4, 2, 0]), array([2, 2, 1, 1, 0]))

In [506]:
_, idx, inv = np.unique([1, 1, 3, 3, 0], return_inverse=True, return_index=True)
dd = {i: inv[base] for i, base in enumerate(np.argsort(idx))}
print(dd)
print(inv)
#[dd[i] for i in inv]

{0: 1, 1: 2, 2: 1}
[1 1 2 2 0]


In [475]:
_, idx, inv

(array([0, 1, 3]), array([4, 0, 2]), array([1, 1, 2, 2, 0]))

In [499]:
np.argsort(idx)[::-1], idx

(array([0, 2, 1]), array([4, 0, 2]))

In [ ]:
for idx, base in enumerate([1, 1, 0, 0, 2]):
    
    bidx = [2, 0, 4].index(idx)
    
    

In [420]:
np.unique(["A", "B", "C", "C", "B"], return_inverse=True)

(array(['A', 'B', 'C'], dtype='<U1'), array([0, 1, 2, 2, 1]))

In [424]:
a, b, c = np.unique(["B", "A", "C", "C", "A"], return_inverse=True, return_index=True)
a,b,c

(array(['A', 'B', 'C'], dtype='<U1'), array([1, 0, 2]), array([1, 0, 2, 2, 0]))

In [432]:
tuple(list(b).index(i) for i in c)

(0, 1, 2, 2, 1)

In [437]:
np.mat

/tmp/ipykernel_590837/1517603257.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.where(b == c)


(array([], dtype=int64),)

In [430]:
np.unique(data.values, return_inverse=True, return_index=True, return_counts=True, axis=1)

(array([[0, 0, 0, 0, 1, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 1, 1, 0, 1, 1],
        [0, 0, 0, 1, 1, 0]]),
 array([6, 5, 1, 2, 3, 0]),
 array([5, 2, 3, 4, 4, 1, 0]),
 array([1, 1, 1, 1, 2, 1]))

In [428]:
data

,t0,t1,t2,t3,t4,t5,t6
alpha,1,0,0,1,1,0,0
beta,0,0,1,0,0,0,0
gamma,1,1,0,0,0,0,0
delta,1,1,0,1,1,1,0
epsilon,0,0,1,1,1,0,0


In [396]:
np.unique(data.values, axis=1, return_counts=True)

(array([[0, 0, 0, 0, 1, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 1, 1, 0, 1, 1],
        [0, 0, 0, 1, 1, 0]]),
 array([1, 1, 1, 1, 2, 1]))

In [392]:
np.unique(data.values[:, 0], return_inverse=True)

(array([0, 1]), array([1, 0, 1, 1, 0]))

In [376]:
np.unique(data.values[0], return_inverse=True)

(array([0, 1]), array([1, 0, 0, 1, 1, 0, 0]))

In [368]:
preprocess_matrix(data)

(1,
 {(1, 0, 1, 1, 0): 1,
  (0, 0, 1, 1, 0): 1,
  (0, 1, 0, 0, 1): 1,
  (1, 0, 0, 1, 1): 2})

In [351]:
np.vstack(list({tuple(row) for row in data.values}))

array([[1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 1, 0, 1, 1, 1, 0]])

In [320]:
data

,t0,t1,t2,t3,t4,t5,t6
alpha,1,0,0,1,1,0,0
beta,0,0,1,0,0,0,0
gamma,1,1,0,0,0,0,0
delta,1,1,0,1,1,1,0
epsilon,0,0,1,1,1,0,0


In [675]:
data = pd.DataFrame(
    #columns=["t" + str(i) for i in range(7)],
    index=["alpha", "beta", "gamma", "delta", "epsilon"],
    data=[
        [1, 0, 0, 1, 1 ,0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 1, 1, 1, 0],
        [0, 0, 1, 1, 1, 0, 0],
    ],
)

In [678]:
data

,0,1,2,3,4,5,6
alpha,1,0,0,1,1,0,0
beta,0,0,1,0,0,0,0
gamma,1,1,0,0,0,0,0
delta,1,1,0,1,1,1,0
epsilon,0,0,1,1,1,0,0


In [282]:
np.array(data)

array([[1, 0, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 0, 0]])

### SPR

In [49]:
tree.treenode.idx, tree.nnodes

(8, 9)

In [200]:
np.random.default_rng().choice(list({1,2,3}))

1

In [254]:

def move_spr(
    tree: toytree.ToyTree, 
    seed: int=None, 
    inplace: bool=False,
    ) -> toytree.ToyTree:
    """Return a rooted ToyTree one SPR move from the current tree."""
    
    tree = tree if inplace else tree.copy()
    rng = np.random.default_rng(seed)

    # randomly select a subtree idx (any non-root Node)
    sidx = rng.choice(tree.nnodes - 1)
    print(sidx)
    
    # get list of Nodes (edges) where subtree can be inserted. This 
    # cannot be the root, or a descendant on the subtree Node, or the
    # subtree itself.
    edges = (
        set(range(tree.nnodes - 1)) - 
        set([i.idx for i in tree[sidx]._iter_descendants()]) - 
        set([i.idx for i in tree[sidx]._iter_sisters()]) - 
        set([sidx])
    )
    
    # randomly select an edge by its descendant Node
    eidx = rng.choice(list(edges))
    print(eidx)
    
    # detach the selected Node and attach to eidx edge
    tree[sidx]._up._remove_child(tree[sidx])
    tree[sidx]._up = tree[eidx]

    # randomly select point on edge to insert at.
    # ...
    
    # update idxs
    tree._update()
    return tree
    
move_spr(tree).draw('s')

3
5


(<toyplot.canvas.Canvas at 0x7fe88b5ee910>,
 <toytree.core.drawing.toytree_mark.ToytreeMark at 0x7fe88b65b130>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="285.0px" height="275.0px" viewBox="0 0 285.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t062df47094be420aac76a565768d45f3"> 0 1 2 3 4 5 6 7 alpha delta gamma epsilon

In [66]:
tree.draw('s')#layout='unroot');
tree.mod.drop_tips("gamma").draw('s')#layout='unroot');

(<toyplot.canvas.Canvas at 0x7fe8dce822b0>,
 <toytree.core.drawing.toytree_mark.ToytreeMark at 0x7fe8dd44ca30>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="285.0px" height="275.0px" viewBox="0 0 285.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3b739d5d421d45f5875e456eabb3425d"> 0 1 2 3 4 5 6 7 8 alpha delta gamma beta epsilon

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="285.0px" height="275.0px" viewBox="0 0 285.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4fd933f787e14ad4b5302560e8f51d71"> 0 1 2 3 4 5 6 beta epsilon alpha delta

In [62]:
tree.get_edges()

,child,parent
0,0,5
1,1,5
2,2,6
3,3,7
4,4,7
5,5,6
6,6,8
7,7,8


In [25]:
tree.draw(ts='s', layout='unroot', node_sizes=25, node_markers='s', node_labels="fitch", node_mask=False);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="285.0px" viewBox="0 0 350.0 285.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7904d69aa26f457c97a6ae731e143a4a"> {1} {1} {1} {0} {0} {1} {1} {0} {0, 1} alpha delta gamma beta epsilon

### Vertebrate tree

In [79]:
newick = ("(((((mouse, chimpanzee),((pigeon, crocodile),lizard)),salamander),trout), lamprey);")
tree = toytree.tree(newick).mod.edges_extend_tips_to_align()
tree.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td3e49ed1497847bfa465d52bc59872a1"> mouse chimpanzee pigeon crocodile lizard salamander trout lamprey

In [ ]:
tree.set_node_data()

### Start with an unrooted tree hypothesis

In [147]:
import pandas as pd
import numpy as np

data = pd.DataFrame(
	columns=["t" + str(i) for i in range(6)],
	index=["alpha", "beta", "gamma", "delta", "epsilon"],
	data=[
		[1, 0, 0, 1, 1 ,0],
		[0, 0, 1, 0, 0, 0],
		[1, 1, 0, 0, 0, 0],
		[1, 1, 0, 1, 1, 1],
		[0, 0, 1, 1, 1, 0],
	],
)
print(data.to_markdown())

|         |   t0 |   t1 |   t2 |   t3 |   t4 |   t5 |
|:--------|-----:|-----:|-----:|-----:|-----:|-----:|
| alpha   |    1 |    0 |    0 |    1 |    1 |    0 |
| beta    |    0 |    0 |    1 |    0 |    0 |    0 |
| gamma   |    1 |    1 |    0 |    0 |    0 |    0 |
| delta   |    1 |    1 |    0 |    1 |    1 |    1 |
| epsilon |    0 |    0 |    1 |    1 |    1 |    0 |


In [135]:
tree = toytree.rtree.unittree(6, seed=123)

In [136]:
tree.draw(layout='unroot');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t28f8c805544445ce9b3b2766747b7a93"> r0 r1 r2 r3 r4 r5

### Randomly root on any edge in the tree

In [137]:
tree = toytree.rtree.rtree(12, seed=123)

In [138]:
tree.distance.get_treedist_rf(tree, normalize=True)

0.0

In [139]:
import pandas as pd

pd.DataFrame(tree._iter_bipartitions())

,0,1
0,"(r1, r2)","(r0, r10, r11, r3, r4, r5, r6, r7, r8, r9)"
1,"(r3, r4)","(r0, r1, r10, r11, r2, r5, r6, r7, r8, r9)"
2,"(r1, r2, r3, r4)","(r0, r10, r11, r5, r6, r7, r8, r9)"
3,"(r5, r6)","(r0, r1, r10, r11, r2, r3, r4, r7, r8, r9)"
4,"(r0, r10, r11, r7, r8, r9)","(r1, r2, r3, r4, r5, r6)"
5,"(r10, r11, r7, r8, r9)","(r0, r1, r2, r3, r4, r5, r6)"
6,"(r7, r8)","(r0, r1, r10, r11, r2, r3, r4, r5, r6, r9)"
7,"(r10, r11)","(r0, r1, r2, r3, r4, r5, r6, r7, r8, r9)"
8,"(r10, r11, r9)","(r0, r1, r2, r3, r4, r5, r6, r7, r8)"


In [24]:
from typing import Optional
import random

def get_random_tree(ntips: int, seed: Optional[int] = None) -> "Node":
    """Return a ladder-like tree of Node objects with 'ntips' tip Nodes.

    Parameters
    ----------
    ntips: int
        The number of tip Nodes that must exist before the tree is returned.

    Returns
    -------
    Node
        The root Node of the set of connected Nodes is returned.
    """
    # set random seed
    random.seed(seed)

    # create root Node with name=root
    root = Node(name="root", dist=0)

    # store Nodes in a list that are currently tips
    tips = [root]

    # add tip Nodes by splitting 'node' into a bifurcation until ntips exist.
    for idx in range(1, ntips):

        # randomly sample one of the current tip Nodes
        node = random.choice(tips)
        tips.remove(node)

        # create two new tips from 'node' and add to tips list  
        child_l = node.add_child(name=f"left{idx}", dist=1)
        child_r = node.add_child(name=f"right{idx}", dist=1)
        tips.extend([child_l, child_r])

    # return root Node that now has ntips descended tip Nodes
    return root


## CLAASSS

In [39]:
from typing import Tuple, Iterator
import pandas as pd


class Node:
    """A simple Node class with private attributes to store Node coordinates."""
    def __init__(self, name: str="", dist: float=1.):
        self.name = name
        self.dist = dist

        # attributes to be updated.
        self.up: Optional['Node'] = None
        self.children: Tuple['Node'] = ()
        self.idx: int = 0
        self.height: float = 0.
        self.x: float = 0.

    def add_child(self, name: str="", dist: float=0.) -> "Node":
        """Add a Node as a child to this one."""
        new_node = Node(name=name, dist=dist)
        new_node.up = self
        self.children += (new_node,)
        return new_node            

    def is_leaf(self) -> bool:
        """return True if Node has no children (is a leaf)."""
        return not self.children

class Tree:
    """A simple Tree class that caches traversal order of connected Nodes"""
    def __init__(self, treenode: 'Node'):
        self.treenode = treenode

        # attributes to be filled.
        self.ntips: int = 0
        self.nnodes: int = 0
        self._idx_dict: Dict[int, 'Node'] = {}

        # cache Node idxorder info
        self._update()
        
    def __iter__(self) -> Iterator[Node]:
        """ToyTree is iterable, returning Nodes in idx order."""
        return (self[i] for i in range(self.nnodes))

    def __getitem__(self, idx: int) -> Node:
        """ToyTree is indexable by idx label to access Nodes."""
        return self._idx_dict[idx]

    def __repr__(self) -> str:
        """Short object representation for toytree.core.tree.ToyTree"""
        return f"<toytree.ToyTree at {hex(id(self))}>"            

    def _update(self) -> None:
        """Set and cache Node idxorder and coordinates.

        idxorder traversal is used to fill the ToyTree._idx_dict to 
        make Nodes easily indexable. While doing this it also calculates
        Node heights and spacing (x) and counts nnodes and ntips.
        """
        # clear depth counters used to get heights during traversal
        depths = {self.treenode: 0}

        # queue starts with root children, and stack starts with root.
        queue = list(self.treenode.children)
        inner_stack = [self.treenode]
        outer_stack = []

        # traverse left then right subtrees to fill and pull from queue
        while queue:
            # get node from start of queue to proceed levelorder
            node = queue.pop()

            # set depth of this node from the root
            depths[node] = depths[node.up] + node.dist

            # if leaf add to output stack and update farthest depth
            if node.is_leaf():
                outer_stack.append(node)
            else:
                inner_stack.append(node)

            # add node's children to the queue (left child on end)
            queue.extend(node.children)

        # get max_depth from root, height is measured relative to this.
        max_depth = max(depths.values())

        # clear idx cache and counter to be filled next
        idx = 0
        self._idx_dict.clear()

        # return nodes in reverse order they were added to stack
        while outer_stack:
            node = outer_stack.pop()
            node.height = max_depth - depths[node]
            node.x = idx
            node.idx = idx
            self._idx_dict[idx] = node
            idx += 1
        self.ntips = idx

        # return internal nodes..
        while inner_stack:
            node = inner_stack.pop()
            node.height = max_depth - depths[node]            
            node.x = sum(i.x for i in node.children) / 2            
            node.idx = idx
            self._idx_dict[idx] = node            
            idx += 1
        self.nnodes = idx


    def get_node_coordinates(self) -> pd.DataFrame:
        """Return a DataFrame with Node coordinates for down-facing tree."""
        iter_coords = ((i.x, i.height) for i in tree)
        return pd.DataFrame(data=iter_coords, columns=('x', 'y'))

    def get_edges(self) -> pd.DataFrame:
        """Return a DataFrame with edges of the tree."""
        edges = ((i.idx, i.up.idx) for i in self if i.up)
        return pd.DataFrame(data=edges, columns=("child", "parent"))


In [40]:
root_node = get_random_tree(20)
tree = Tree(root_node)
# tree.get_node_coordinates()

In [131]:
#
from scipy.linalg import expm

expm([[0, 1], [1, 0]])

array([[1.54308063, 1.17520119],
       [1.17520119, 1.54308063]])

In [113]:
# get the root Node of a collection of Nodes
root_node = get_random_tree(ntips=10, seed=123)

# get a Tree that stores the Node as its root.
tree = Tree(root_node)
print(tree)

In [123]:
# get and print the tip Nodes
print([tree[i].idx for i in range(tree.ntips)])

# get and print the root Node
print(tree.treenode.idx)

# get and print all nodes in idxorder (topogically sorted)
print([tree[i].idx for i in range(tree.nnodes)])

# get and print all nodes in reverse idxorder (topogically sorted)
print([tree[i].idx for i in range(tree.nnodes)[::-1]])

# get a specific Node by its idx label
print(tree[5].idx)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
18
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
5


In [112]:
import toyplot

# create a Canvas and Axes
canvas = toyplot.Canvas(width=450, height=350)
axes = canvas.cartesian(label="toyplot drawing of Nodes and Edges")

# add edges lines
for node in tree:
    if node.up:
        xpos = (coords.x[node.idx], coords.x[node.up.idx])
        ypos = (coords.y[node.idx], coords.y[node.up.idx])
        # e.g., (0, 1) -> (1, 2)
        axes.plot(xpos, ypos, color="black")
        
# add node circles
axes.scatterplot(coords.x, coords.y, size=16)
axes.text(coords.x, coords.y, range(tree.nnodes), color='black')

# canvas will automatically display in jupyter

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="450.0px" height="350.0px" viewBox="0 0 450.0 350.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfc6129348d0d41659487e6e3c9c8906d"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 0 5 10 15 20 0 3 6 9 toyplot drawing of Nodes and Edges

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="350.0px" viewBox="0 0 350.0 350.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t90d02f9454fc495d9f1e374c693e9e88"> 0 5 10 15 20 0 3 6 9 toyplot drawing of Node and Edges

In [32]:
%%timeit
tree._update()

295 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [94]:
tree = toytree.rtree.unittree(10)

In [111]:
tree.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="275.0px" viewBox="0 0 260.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t242045e2cdcb40a6a0b8e90d90d15f59"> r0 r1 r2 r3 r4 r5 r6 r7 r8 r9

In [109]:
[tree[i] for i in range(tree.nnodes)]

[Node(0),
 Node(1),
 Node(2),
 Node(3),
 Node(4),
 Node(5),
 Node(6),
 Node(7),
 Node(8),
 Node(9),
 Node(10),
 Node(11),
 Node(12),
 Node(13),
 Node(14),
 Node(15),
 Node(16),
 Node(17),
 Node(18)]

In [98]:
list(tree.treenode.traverse("levelorder"))

[Node(18),
 Node(16),
 Node(17),
 Node(11),
 Node(15),
 Node(8),
 Node(9),
 Node(0),
 Node(10),
 Node(13),
 Node(14),
 Node(1),
 Node(2),
 Node(12),
 Node(5),
 Node(6),
 Node(7),
 Node(3),
 Node(4)]

In [91]:
root_node = get_random_tree(10, seed=123)

In [92]:
gen = _traverse_levelorder(root_node)

In [93]:
x = list(gen)

['root']
['left1', 'right1']
['right1', 'left3', 'right3']
['left3', 'right3', 'left2', 'right2']
['right3', 'left2', 'right2', 'left5', 'right5']
['left2', 'right2', 'left5', 'right5', 'left4', 'right4']
['right2', 'left5', 'right5', 'left4', 'right4', 'left6', 'right6']
['left5', 'right5', 'left4', 'right4', 'left6', 'right6', 'left8', 'right8']
['right5', 'left4', 'right4', 'left6', 'right6', 'left8', 'right8']
['left4', 'right4', 'left6', 'right6', 'left8', 'right8']
['right4', 'left6', 'right6', 'left8', 'right8']
['left6', 'right6', 'left8', 'right8']
['right6', 'left8', 'right8']
['left8', 'right8', 'left7', 'right7']
['right8', 'left7', 'right7']
['left7', 'right7']
['right7']
['left9', 'right9']
['right9']


In [69]:
for item in gen:
    print(item)

Node(root)
Node(left1)
Node(left3)
Node(left5)
Node(right5)
Node(right3)
Node(left4)
Node(right4)
Node(right1)
Node(left2)
Node(left6)
Node(right6)
Node(left7)
Node(right7)
Node(left9)
Node(right9)
Node(right2)
Node(left8)
Node(right8)


In [41]:
nodes = list(gen)

In [43]:
list(gen)

[]

In [19]:
print([tree[i] for i in range(tree.ntips)])
print([tree[i] for i in range(10)])

# get and print the root Node
print(tree.treenode)

# get and print all nodes in idxorder (topogically sorted)
print([tree[i] for i in range(tree.nnodes)])

# get and print all nodes in reverse idxorder (topogically sorted)
print([tree[i] for i in range(tree.nnodes)[::-1]])

# get a specific Node by its idx label
print(tree[5])

[Node(0), Node(1), Node(2), Node(3), Node(4), Node(5), Node(6), Node(7), Node(8), Node(9), Node(10), Node(11)]
[Node(0), Node(1), Node(2), Node(3), Node(4), Node(5), Node(6), Node(7), Node(8), Node(9)]
Node(idx=22, root)
[Node(0), Node(1), Node(2), Node(3), Node(4), Node(5), Node(6), Node(7), Node(8), Node(9), Node(10), Node(11), Node(12), Node(13), Node(14), Node(15), Node(16), Node(17), Node(18), Node(19), Node(20), Node(21), Node(22)]
[Node(22), Node(21), Node(20), Node(19), Node(18), Node(17), Node(16), Node(15), Node(14), Node(13), Node(12), Node(11), Node(10), Node(9), Node(8), Node(7), Node(6), Node(5), Node(4), Node(3), Node(2), Node(1), Node(0)]
Node(idx=5, leaf)


In [4]:
# tree._old_update()
tree = toytree.rtree.rtree(12, seed=123)
tree.draw('s', layout='d');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="265.0px" viewBox="0 0 350.0 265.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t683d14e25b5d456f8ff81b9c5ad95bc2"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11

In [ ]:
# c        queue-at-end          # inner                        # outer
# 22        [21, 15]              []                             [] 
# 15        [21, 14, 13]          [15]
# 13        [21, 14, 9, 12]       [15, 13]
# 12        [21, 14, 9, 10, 11]   [15, 13, 12]
# 11        [21, 14, 9, 10]       [15, 13, 12]                        [11]
# 10        [21, 14, 9]           [15, 13, 12]                        [11, 10]
# 9         [21, 14]              [15, 13, 12]                        [11, 10, 9]
# 14        [21, 7, 8]            [15, 13, 12, 14]                    [11, 10, 9]
# 8         [21, 7]               [15, 13, 12, 14]                    [11, 10, 9, 8]
# 7         [21]                  [15, 13, 12, 14]                    [11, 10, 9, 8, 7]
# 21        [0, 20]               [15, 13, 12, 14, 21]                ''
# 20        [0, 19, 16]           [15, 13, 12, 14, 21, 20]            ''
# 16        [0, 19, 5, 6]         [15, 13, 12, 14, 21, 20, 16]        ''
# 6         [0, 19, 5]            [15, 13, 12, 14, 21, 20, 16]                  [11, 10, 9, 8, 7, 6]
# 5         [0, 19]               [15, 13, 12, 14, 21, 20, 16]                  [11, 10, 9, 8, 7, 6, 5]
# 19        [0, 18, 17]           [15, 13, 12, 14, 21, 20, 16, 19]              [11-5]
# 17        [0, 18, 3, 4]         [15, 13, 12, 14, 21, 20, 16, 19, 17]          [11-5]
# 4         [0, 18, 3]             ''                                           [11-4]
# 3         [0, 18]                ''                                           [11-3]
# 18        [0, 1, 2]             [15, 13, 12, 14, 21, 20, 16, 19, 17, 18]
# 2         [0, 1]
# 1         [0]
# 0         []                                                                  [11-0]


# USING [::-1]
# c        queue-at-end          # inner                        # outer
# 22        [15, 21]              []                             [] 
# 21        [15, 20, 0]           [21]                           []
# 0         [15, 20]              [21]                           [0]
# 20        [15, 16, 19]          [21, 20]                       [0]
# 19        [15, 16, 17, 18]      [21, 20, 19]                   [0]
# 18        [15, 16, 17, 2, 1]    [21, 20, 19, 18]               [0]
# 1         [15, 16, 17, 2]       [21, 20, 19, 18]               [0, 1]
# 2         [15, 16, 17]          [21, 20, 19, 18]               [0, 1, 2]
# 17        [15, 16, 6, 5]        [21, 20, 19, 18, 17]           [0, 1, 2]

In [28]:
toytree.tree("((a,b:2),((d,c:5),e));").draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9120c3afb3284cada8fd849a299ee7d4"> a b d c e

In [8]:
list(tree._new_update())

[Node(0),
 Node(1),
 Node(2),
 Node(3),
 Node(4),
 Node(5),
 Node(6),
 Node(7),
 Node(8),
 Node(9),
 Node(10),
 Node(11),
 Node(12),
 Node(13),
 Node(14),
 Node(15),
 Node(16),
 Node(17),
 Node(18),
 Node(19),
 Node(20),
 Node(21),
 Node(22)]

In [8]:
# tree._old_update()
tree.draw('s', layout='d');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="265.0px" viewBox="0 0 350.0 265.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc5c7378bb164481da5e439c301df422e"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11

In [ ]:
# [22, 21, x, 20, 19, 18, x, x, 17, x, x, 16, x, x, 15, 14, x, x, 13, x, 12, x, x, ]
# []
# [0, 1, 2, 3, ... 11, ]

In [ ]:
[22, 21, 20, x, x, 19, x, 18, x, x, 17, x, 16, 15, x, x, 14, 13, x, x, 12, x, x]
[20, 18, 19, 21, 15, 13, 12, 14, 16, 17, 22]



In [11]:
# tree._old_update()
tree.mod.ladderize().draw('s', layout='d');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="265.0px" viewBox="0 0 350.0 265.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t82ad9d3ed8614e4d924f6f03aae4cd7f"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r4 r3 r2 r1 r0 r11 r6 r5 r8 r7 r9 r10

In [22]:
        # clear counters to count up idxs and depth/heights from.
        idx = 0
        max_depth = 0
        self._idx_dict.clear()
        self.treenode._depth = 0

        # queue starts with root children, and stack starts with root.
        queue = list(self.treenode._children[::-1])
        inner_stack = [self.treenode]
        outer_stack = deque([])

        # traverse left then right subtrees to fill and pull from queue
        while queue:
            # get node from end of the queue
            node = queue.pop()
            
            # set depth of this node from the root
            node._depth = node._up._depth + node._dist
            
            # if leaf add to output stack and update farthest depth
            if node.is_leaf():
                outer_stack.append(node)
                max_depth = max(max_depth, node._depth)
                node._idx = idx
                node._x = idx
                idx += 1
            else:
                inner_stack.append(node)

            # add node's children to the queue (left child on end)
            queue.extend(node._children[::-1])

        # return nodes in reverse order they were added to stack
        while outer_stack:
            node = outer_stack.popleft()
            node._height = max_depth - node._depth
            self._idx_dict[node._idx] = node
        self.ntips = idx
        while inner_stack:
            node = inner_stack.pop()
            node._height = max_depth - node._depth
            node._x = sum(i._x for i in node._children) / 2            
            node._idx = idx
            self._idx_dict[idx] = node            
            idx += 1
        self.nnodes = idx

NameError: name 'self' is not defined

In [5]:
%%timeit
tree._update()

22.3 µs ± 98.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
idx = 3
node = tree[0]

In [14]:
node._children

()

In [12]:
%%timeit
node.idx

66.3 ns ± 1.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [5]:
list(tree._new_traverse_idxorder())
tree.get_edges()
tree.mod.ladderize().draw('s');

AttributeError: 'ToyTree' object has no attribute '_new_traverse_idxorder'

In [6]:
tree.mod.ladderize().draw('s');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="255.0px" height="275.0px" viewBox="0 0 255.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t93eed0d19bff47669f9b84da0a94beeb"> 0 1 2 3 4 a b c

In [11]:
from collections import deque
        
def new_idxorder(self):
    idx = 0
    max_depth = 0
    self._depth = 0
    queue = list(self.children)
    inner_stack = [self]
    outer_stack = deque([])
    while queue:
        # get node from end of the queue
        node = queue.pop()
        
        # set depth of this node from the root
        node._depth = node._up._depth + node._dist
        
        # if leaf add to output stack and update farthest depth
        if node.is_leaf():
            outer_stack.append(node)
            max_depth = max(max_depth, node._depth)
            #node._idx = idx
            node._x = idx
            idx += 1
        else:
            inner_stack.append(node)
            node._x = sum(i._x for i in node.children) / 2

        # add node's children to the queue (left child on end)
        queue.extend(node.children)

    # return nodes in reverse order they were added to stack
    while outer_stack:
        node = outer_stack.popleft()
        node._height = max_depth - node._depth
        yield node
    while inner_stack:
        node = inner_stack.pop()
        node._height = max_depth - node._depth
        #node._idx = idx
        yield node
        idx += 1


In [12]:
#%%timeit
list(new_idxorder(tree.treenode))

[Node(0),
 Node(1),
 Node(2),
 Node(3),
 Node(4),
 Node(5),
 Node(6),
 Node(7),
 Node(8),
 Node(9),
 Node(10),
 Node(11),
 Node(12),
 Node(13),
 Node(14),
 Node(15),
 Node(16),
 Node(17),
 Node(18),
 Node(19),
 Node(20),
 Node(21),
 Node(22)]

In [15]:
tree._get_node_coordinates()

array([[5.        , 0.66666667],
       [4.        , 0.66666667],
       [3.        , 0.66666667],
       [2.        , 0.66666667],
       [1.        , 0.        ],
       [0.        , 0.        ],
       [4.5       , 1.        ],
       [2.5       , 1.        ],
       [0.5       , 0.66666667],
       [2.5       , 1.33333333]])

In [414]:
for node in tree.traverse("idxorder"):
    print(node.idx, node.height)

0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.6
13 0.6
14 0.8
15 0.3999999999999999
16 0.6
17 0.3999999999999999
18 0.19999999999999996
19 0.3999999999999999
20 0.6
21 0.8
22 1.0


In [359]:
%%timeit

max_dist = 0
for node in tree.traverse("preorder"):
    if node._up:
        node._height = node._dist + node._up._height
    else:
        node._height = 0
    if node.is_leaf():
        max_dist = max(max_dist, node._height)

107 µs ± 436 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [334]:
%%timeit
list(old_idxorder(tree.treenode))

9.02 µs ± 76.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [324]:
[22, 14, 12, 13, 21, 16, 15, 20, 17, 19, 18]

SyntaxError: invalid syntax (3981575886.py, line 1)

In [308]:
tree.draw('s', height=400);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="265.0px" height="400.0px" viewBox="0 0 265.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc2ad17db10bd4414a550cd54a6545915"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11

In [283]:
list(idxorder(tree.treenode))

[Node(0),
 Node(1),
 Node(2),
 Node(3),
 Node(4),
 Node(5),
 Node(6),
 Node(7),
 Node(8),
 Node(9),
 Node(10)]

In [151]:
tree = tree.root(2)
tree.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="265.0px" height="275.0px" viewBox="0 0 265.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1f6773af11984c10b72378499cefea95"> r2 r1 r3 r4 r5 r6 r0 r7 r8 r9 r10 r11

In [25]:
import toytree

# a six tip tree
tree = toytree.tree("(((alpha,delta),gamma),(beta,epsilon));")
tree.mod.edges_extend_tips_to_align(inplace=True)

# an example binary trait for species alpha-epsilon
trait = {
    "alpha": 1,
    "beta": 0,
    "delta": 1,
    "epsilon": 0,
    "gamma": 1,
}

# set traits on the tree to make them easy to color
tree.set_node_data(
    feature="trait",
    mapping=trait,
    inplace=True,
)

In [26]:
print(tree.get_node_data(['name', "trait"]).to_markdown())

|    | name    | trait   |
|---:|:--------|:--------|
|  0 | alpha   | 1       |
|  1 | delta   | 1       |
|  2 | gamma   | 1       |
|  3 | beta    | 0       |
|  4 | epsilon | 0       |
|  5 |         |         |
|  6 |         |         |
|  7 |         |         |
|  8 |         |         |


In [27]:
c, a, m = tree.draw(
    tree_style='s',
    use_edge_lengths=False, 
    tip_labels_style={"font-size": 18},
    node_labels="trait",
    node_colors=[
        toytree.color.COLORS1[i] if i !="" else "lightgrey" 
        for i in tree.get_node_data("trait")
    ],
);

import toyplot.svg
c.style["background-color"] = "white"
# toyplot.svg.render(c, "./fig-6.1.svg")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="285.0px" height="275.0px" viewBox="0 0 285.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:white;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t333e903837df460a8e5f809914bf5ce8"> 1 1 1 0 0 alpha delta gamma beta epsilon

In [47]:
import toyplot

# create a canvas and cartesian axes
canvas = toyplot.Canvas(width=600, height=300)
ax0 = canvas.cartesian(show=False, grid=(1, 2, 0))
ax1 = canvas.cartesian(show=False, grid=(1, 2, 1))

# make two copies of tree, each filled w/ ancestral states but diff root.
tree1 = tree.set_node_data(
    "trait",
    mapping={8: 1, 7: 0},
    inherit=True,
)
tree2 = tree.set_node_data(
    "trait",
    mapping={8: 0, 6: 1},
    inherit=True,
)

# draw the tree with colored Node states
tree1.draw(
    axes=ax0,
    tree_style='s',
    tip_labels_style={"font-size": 18},
    node_labels="trait",
    node_colors=[
        toytree.color.COLORS1[i] if i !="" else "lightgrey" 
        for i in tree1.get_node_data("trait")
    ],
);

# add an X marker to show the transition event above a Node
node = tree1.get_mrca_node('epsilon', 'beta')
ax0.scatterplot(-node.height, node._x, marker="x", size=20, 
                mstyle={"stroke-width": 3}, color=toytree.color.COLORS1[0])

# draw the tree with colored Node states
tree2.draw(
    axes=ax1,
    tree_style='s',
    tip_labels_style={"font-size": 18},
    node_labels="trait",
    node_colors=[
        toytree.color.COLORS1[i] if i !="" else "lightgrey" 
        for i in tree2.get_node_data("trait")
    ],
);

# add an X marker to show the transition event above a Node
node = tree2.get_mrca_node('gamma', 'delta', 'alpha')
ax1.scatterplot(-node.height - node.dist / 2, node._x, marker="x", size=20, 
                mstyle={"stroke-width": 3},  color=toytree.color.COLORS1[1])

canvas.style["background-color"] = "white"
toyplot.svg.render(canvas, "./fig-6.2.svg")
canvas

In [ ]:
c, a, m = tree.draw(
    tree_style='s',
    use_edge_lengths=False, 
    tip_labels_style={"font-size": 18},
    node_labels="trait",
    node_colors=[
        toytree.color.COLORS1[i] if i !="" else "lightgrey" 
        for i in tree.get_node_data("trait")
    ],
);



import toyplot.svg
# toyplot.svg.render(c, "./fig-6.1.svg")

In [231]:
tree = tree.unroot()
tree.draw(
    ts='s', 
    use_edge_lengths=False, 
    #layout='unroot', 
    node_colors=tree.get_node_data("character"),
    tip_labels_style={"font-size": 18});

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="285.0px" height="275.0px" viewBox="0 0 285.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t157120519c794dd19aa8f6e9805b6d6f"> 0 1 2 3 4 5 6 7 alpha delta gamma beta epsilon

### Get tip data for a given trait/site

In [60]:
site = ["C", "A", "G", "A", "C"]

### Set data on tree Nodes

In [61]:
tree = tree.set_node_data(
    feature="trait",
    mapping={i: {j} for (i, j) in enumerate(site)},
    default="-"
)

print(tree.get_node_data("trait"))

0    {C}
1    {A}
2    {G}
3    {A}
4    {C}
5      -
6      -
7      -
8      -
Name: trait, dtype: object


### Draw data on the tree

In [62]:
tree.draw(
#     layout='unr',
    width=450,
    height=400,
    use_edge_lengths=False,
    node_sizes=20, 
    node_mask=False,
    node_colors="lightgrey",
    node_markers="r2x1",
    node_labels=tree.get_node_data("trait"),
    node_labels_style={"font-size": 14},
    tip_labels=False,
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="450.0px" height="400.0px" viewBox="0 0 450.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc3ad632a4ba2441fb3ade6fa51ad807c"> {'C'} {'A'} {'G'} {'A'} {'C'} - - - -

### Perform postorder traversal

In [94]:
tree.get_node_data("")

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
Name: idx, dtype: object

In [91]:
range(5)[::-1]

range(4, -1, -1)

In [69]:
from typing import Tuple, Collection

def fitch_parsimony_example(tree: toytree.ToyTree) -> int:
    """Return Fitch parsimony score given a tree with data 'state'. 
    
    This function will update the .state attribute of each 
    internal Node on the tree, which can be visualized afterwards.
    
    Parameters
    ----------
    tree: ToyTree
        A tree with data 'state' on each tip Node rom which to
        calculate the Fitch parsimony score. 
        
    Returns
    -------
    int
        The Fitch parsimony score as minimum number of changes
        required for this data to evolve on the tree.
    """
    nchanges = 0
    for node in tree.traverse("postorder"):
        if not node.is_leaf():

            # start by setting an empty set for this node.
            node.trait = set()

            # get the intersection of its children
            shared = set.intersection(*(i.trait for i in node.children))

            # set shared state set as ancestral if it exists
            if shared:
                node.trait = shared

            # else, store the union and increment change counter
            else:
                node.trait = set.union(*(i.trait for i in node.children))
                nchanges += 1
    return nchanges

In [70]:
fitch_parsimony_example(tree)

3

In [275]:
tree.draw(
    width=450, 
    height=400,
    node_sizes=20, 
    node_mask=False,
    node_colors="lightgrey",
    node_markers="r2x1",
    node_labels=tree.get_node_data("state"),
    tip_labels=False,
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="450.0px" height="400.0px" viewBox="0 0 450.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td570fc7705324d6a8eef1a33d4c737d2"> {'C'} {'A'} {'C'} {'A'} {'C'} {'G'} {'A', 'C'} {'G', 'C'} {'C'} {'A', 'C'} {'C'}

In [24]:
c, a, m = tree.draw(scale_bar=True, width=400)
a.text(
    np.repeat(0.25, tree.ntips),
    np.arange(tree.ntips),
    [str(i) for i in tree.get_tip_data('state')],
    style={"text-anchor": 'start', 'fill': 'black'}
)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="275.0px" viewBox="0 0 400.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9c1888d48f8646f1827fc5c803c33b40"> r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 ['-', 0, '-', '-'] ['-', 0, '-', '-'] ['-', 0, '-', '-'] ['-', 0, '-', '-'] ['-', 0, '-', '-'] ['-', 0, '-', '-'] ['-', 0, '-', '-'] ['-', 0, '-', '-'] ['-', 0, '-', '-'] ['-', 0, '-', '-'] 1 0.5 0